In [0]:
# 구글 Drive와 연동

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()


# 구글 Drive 마운트

from google.colab import drive

drive.mount('/content/gdrive')

# 폴더 경로 지정

from os import path

import sys

sys.path.insert(0, '/content/gdrive/My Drive/Colab_Notebooks/')


# GPU 설정 및 cuda, pytorch 설치

from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl torchvision

##########################

#  그 외 필요한 package 설치


# github에서 dataset 다운로드(필요한 경우)
! git clone https://github.com/aweek43/dataset.git

! ls


#########################

# Main Code

from preprocessing import *
import time

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models

lr = 0.001
device = torch.device("cuda")

train_loader = trainDataset()
test_loader = testDataset()

model = models.densenet121(pretrained=True).cuda()
model.classifier = nn.Linear(1024, 28).cuda()

########################
model.load_state_dict(torch.load('gdrive/My Drive/Colab_Notebooks/checkpoint/checkpoint_epoch7.pt'))
model.eval()
########################
criterion = nn.BCEWithLogitsLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

def train(epoch):
    model.train()
    for idx, (data, filename) in enumerate(train_loader):
          
        target = filename
        data, target = data[0].to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if idx % 10 == 0:
            print("epoch: ", epoch, "  process: ", int((idx / len(train_loader)) * 100),
                "%  Loss: ", loss.data.item())

    torch.save(model.state_dict(), 'gdrive/My Drive/Colab_Notebooks/checkpoint/checkpoint_epoch'+str(epoch)+'.pt')

def test():
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, filename in test_loader:
            target = filename

            data, target = data[0].to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).data.item()
    
    test_loss /= len(test_loader.dataset)
    print("Average Loss: ", test_loss)


if __name__ == "__main__":
    for epoch in range(8, 150):
        start = time.time()
        train(epoch)
        end = time.time()
        print("It takes ", end - start, " seconds")
        if epoch % 3 == 0:
          test()



#########################

E: Package 'python-software-properties' has no installation candidate
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'dataset' already exists and is not an empty directory.
adc.json  dataset  gdrive  sample_data
epoch:  8   process:  0 %  Loss:  0.21342240273952484
epoch:  8   process:  1 %  Loss:  0.1878340095281601
epoch:  8   process:  2 %  Loss:  0.196318581700325
epoch:  8   process:  3 %  Loss:  0.17352642118930817
epoch:  8   process:  4 %  Loss:  0.15299110114574432
epoch:  8   process:  5 %  Loss:  0.18660369515419006
epoch:  8   process:  6 %  Loss:  0.17233161628246307
epoch:  8   process:  7 %  Loss:  0.17780226469039917
epoch:  8   process:  8 %  Loss:  0.21169987320899963
epoch:  8   process:  9 %  Loss:  0.19313761591911316
epoch:  8   process:  10 %  Loss:  0.19370180368423462
epoch:  8   process:  12 %  Loss:  0.19405147433280945
epoch:  8   process:  13 %  Loss: